In [1]:
# Install necessary packages
!pip install torch
!pip install transformers wandb easydict iterative-stratification tqdm scikit-learn pandas matplotlib




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [3]:
#mount google drive to access data and code
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#create link to data folder
!ln -s /content/drive/MyDrive/DescEmb_Data/ /content/DescEmb_Data


In [8]:
cd /content/drive/MyDrive/DescEmb/

/content/drive/MyDrive/DescEmb


In [ ]:
#preprocess using source data mimiciii
!python preprocess/preprocess_main.py --src_data mimiciii --dataset_path /content/drive/MyDrive/DescEmb_Data/data_input_path/mimic/ --dest_path /content/drive/MyDrive/DescEmb_Data/data_output_path/ --ccs_dx_tool_path /content/drive/MyDrive/DescEmb_Data/data_input_path/ccs_multi_dx_tool_2015.csv

working directory .. :  /content/drive/MyDrive/DescEmb
Destination directory is set to /content/drive/MyDrive/DescEmb_Data/data_output_path/
Data directory is set to /content/drive/MyDrive/DescEmb_Data/data_input_path/mimic/
mimic_cohort.pkl already exists skip create cohort step!___
data preparation initialization .. mimiciii lab
df_load ! .. mimiciii lab
data preparation initialization .. mimiciii med
df_load ! .. mimiciii med
data preparation initialization .. mimiciii inf
^C


In [ ]:
#Pre-train a DescEmb model with Masked Language Modeling (MLM) .
#Input path should be the dest path from preprocessing step
#source data- mimiciii , task- mlm, model- descemb_bert
!python main.py --distributed_world_size 1 --input_path /content/drive/MyDrive/DescEmb_Data/data_output_path/mimic --src_data mimiciii --task mlm --model descemb_bert

Streaming output truncated to the last 5000 lines.
[2025-04-27 20:08:18,838][trainers.trainer][INFO] - Finished saving checkpoint to checkpoints/checkpoint_last.pt
[2025-04-27 20:08:18,838][trainers.trainer][INFO] - begin training epoch 287
  0% 0/2 [00:00<?, ?it/s]Epoch 287, Batch 0 loaded...
 50% 1/2 [00:00<00:00,  2.74it/s]Epoch 287, Batch 1 loaded...
100% 2/2 [00:00<00:00,  4.62it/s]
[2025-04-27 20:08:19,272][train][INFO] - epoch: 287, loss: 0.454, auroc: 0.000, auprc: 0.000
[2025-04-27 20:08:19,272][trainers.trainer][INFO] - Saving checkpoint to checkpoints/checkpoint_last.pt
[2025-04-27 20:08:20,177][trainers.trainer][INFO] - Finished saving checkpoint to checkpoints/checkpoint_last.pt
[2025-04-27 20:08:20,177][trainers.trainer][INFO] - begin training epoch 288
  0% 0/2 [00:00<?, ?it/s]Epoch 288, Batch 0 loaded...
 50% 1/2 [00:00<00:00,  2.77it/s]Epoch 288, Batch 1 loaded...
100% 2/2 [00:00<00:00,  4.66it/s]
[2025-04-27 20:08:20,608][train][INFO] - epoch: 288, loss: 0.337, auroc:

In [ ]:
#Train a new DescEmb model:
#Input path should be the dest path from preprocessing step
#source data- mimiciii , task- mortality, model- descemb_rnn, value_mode- NV
!python main.py \
    --distributed_world_size 1 \
    --input_path /content/drive/MyDrive/DescEmb_Data/data_output_path/mimic \
    --model ehr_model \
    --embed_model descemb_rnn \
    --pred_model rnn \
    --src_data mimiciii \
    --ratio 100 \
    --value_mode NV \
    --task mortality ;

2025-04-27 20:57:28 | INFO numexpr.utils NumExpr defaulting to 8 threads.)))
[2025-04-27 20:57:30,285][trainers.trainer][INFO] - {'batch_size': 128,
 'bert_model': 'bert_tiny',
 'device_ids': [0],
 'disable_validation': False,
 'distributed_world_size': 1,
 'dropout': 0.3,
 'embed_model': 'descemb_rnn',
 'enc_embed_dim': 128,
 'enc_hidden_dim': 256,
 'eval_data': None,
 'fold': None,
 'init_bert_params': False,
 'init_bert_params_with_freeze': False,
 'input_path': '/content/drive/MyDrive/DescEmb_Data/data_output_path/mimic',
 'load_pretrained_weights': False,
 'lr': 0.0001,
 'max_event_len': 150,
 'mlm_prob': 0.3,
 'model': 'ehr_model',
 'model_path': None,
 'n_epochs': 1000,
 'patience': 5,
 'pred_embed_dim': 128,
 'pred_hidden_dim': 256,
 'pred_model': 'rnn',
 'ratio': '100',
 'rnn_layer': 1,
 'save_dir': 'checkpoints',
 'save_prefix': 'checkpoint',
 'seed': 1,
 'src_data': 'mimiciii',
 'task': 'mortality',
 'transfer': False,
 'valid_subsets': ['valid', 'test'],
 'value_mode': 'NV'

In [ ]:
#Fine-tune a pre-trained DescEmb model on smaller set of pkl files for debugging purpose
#Input path should be the dest path from preprocessing step
#model path should be the checkpoint path from traing step
#source data- mimiciii , task- mortality, embed_model- descemb_bert, model-ehr_model,value_mode- DSVA_DPE
!python main.py \
    --distributed_world_size 1 \
    --input_path /content/drive/MyDrive/DescEmb_Data/data_output_path/mimic  \
    --model_path /content/drive/MyDrive/DescEmb/outputs/2025-04-28/05-04-39/checkpoints/checkpoint_last.pt \
    --load_pretrained_weights \
    --model ehr_model \
    --embed_model descemb_bert \
    --pred_model rnn \
    --src_data mimiciii \
    --ratio 100 \
    --value_mode DSVA_DPE \
    --task mortality

2025-04-28 02:22:29 | INFO numexpr.utils NumExpr defaulting to 8 threads.)))
[2025-04-28 02:22:31,450][trainers.trainer][INFO] - {'batch_size': 128,
 'bert_model': 'bert_tiny',
 'device_ids': [0],
 'disable_validation': False,
 'distributed_world_size': 1,
 'dropout': 0.3,
 'embed_model': 'descemb_bert',
 'enc_embed_dim': 128,
 'enc_hidden_dim': 256,
 'eval_data': None,
 'fold': None,
 'init_bert_params': False,
 'init_bert_params_with_freeze': False,
 'input_path': '/content/drive/MyDrive/DescEmb_Data/data_output_path/mimic',
 'load_pretrained_weights': True,
 'lr': 0.0001,
 'max_event_len': 150,
 'mlm_prob': 0.3,
 'model': 'ehr_model',
 'model_path': '/content/drive/MyDrive/DescEmb/outputs/2025-04-28/05-04-39/checkpoints/checkpoint_last.pt',
 'n_epochs': 1000,
 'patience': 5,
 'pred_embed_dim': 128,
 'pred_hidden_dim': 256,
 'pred_model': 'rnn',
 'ratio': '100',
 'rnn_layer': 1,
 'save_dir': 'checkpoints',
 'save_prefix': 'checkpoint',
 'seed': 1,
 'src_data': 'mimiciii',
 'task': 'm

In [ ]:
#Fine-tune a pre-trained DescEmb model on the actual set of pkl files
#Input path should be the dest path from preprocessing step
#model path should be the checkpoint path from traing step
#source data- mimiciii , task- mortality, embed_model- descemb_bert, model-ehr_model,value_mode- DSVA_DPE
!python main.py \
    --distributed_world_size 1 \
    --input_path /content/drive/MyDrive/DescEmb_Data/data_output_path/mimic  \
    --model_path /content/drive/MyDrive/DescEmb/outputs/2025-04-28/05-57-30/checkpoints/checkpoint_best.pt \
    --load_pretrained_weights \
    --model ehr_model \
    --embed_model descemb_bert \
    --pred_model rnn \
    --src_data mimiciii \
    --ratio 100 \
    --value_mode DSVA_DPE \
    --task mortality

2025-04-28 02:44:30 | INFO numexpr.utils NumExpr defaulting to 8 threads.)))
[2025-04-28 02:44:31,990][trainers.trainer][INFO] - {'batch_size': 128,
 'bert_model': 'bert_tiny',
 'device_ids': [0],
 'disable_validation': False,
 'distributed_world_size': 1,
 'dropout': 0.3,
 'embed_model': 'descemb_bert',
 'enc_embed_dim': 128,
 'enc_hidden_dim': 256,
 'eval_data': None,
 'fold': None,
 'init_bert_params': False,
 'init_bert_params_with_freeze': False,
 'input_path': '/content/drive/MyDrive/DescEmb_Data/data_output_path/mimic',
 'load_pretrained_weights': True,
 'lr': 0.0001,
 'max_event_len': 150,
 'mlm_prob': 0.3,
 'model': 'ehr_model',
 'model_path': '/content/drive/MyDrive/DescEmb/outputs/2025-04-28/05-57-30/checkpoints/checkpoint_best.pt',
 'n_epochs': 1000,
 'patience': 5,
 'pred_embed_dim': 128,
 'pred_hidden_dim': 256,
 'pred_model': 'rnn',
 'ratio': '100',
 'rnn_layer': 1,
 'save_dir': 'checkpoints',
 'save_prefix': 'checkpoint',
 'seed': 1,
 'src_data': 'mimiciii',
 'task': 'm

In [ ]:
#Train a new DescEmb model with higher patience parameter to allow more epochs
#Input path should be the dest path from preprocessing step
#source data- mimiciii , task- mortality, embed_model- descemb_bert, model-ehr_model,value_mode- DSVA_DPE
!python main.py \
    --distributed_world_size 1 \
    --input_path /content/drive/MyDrive/DescEmb_Data/data_output_path/mimic \
    --model ehr_model \
    --embed_model descemb_bert \
    --pred_model rnn \
    --src_data mimiciii \
    --ratio 100 \
    --value_mode DSVA_DPE \
    --task mortality \
    --patience 20 ;

2025-04-28 18:03:03 | INFO numexpr.utils NumExpr defaulting to 2 threads.)))
[2025-04-28 18:03:06,137][trainers.trainer][INFO] - {'batch_size': 128,
 'bert_model': 'bert_tiny',
 'device_ids': [0],
 'disable_validation': False,
 'distributed_world_size': 1,
 'dropout': 0.3,
 'embed_model': 'descemb_bert',
 'enc_embed_dim': 128,
 'enc_hidden_dim': 256,
 'eval_data': None,
 'fold': None,
 'init_bert_params': False,
 'init_bert_params_with_freeze': False,
 'input_path': '/content/drive/MyDrive/DescEmb_Data/data_output_path/mimic',
 'load_pretrained_weights': False,
 'lr': 0.0001,
 'max_event_len': 150,
 'mlm_prob': 0.3,
 'model': 'ehr_model',
 'model_path': None,
 'n_epochs': 1000,
 'patience': 20,
 'pred_embed_dim': 128,
 'pred_hidden_dim': 256,
 'pred_model': 'rnn',
 'ratio': '100',
 'rnn_layer': 1,
 'save_dir': 'checkpoints',
 'save_prefix': 'checkpoint',
 'seed': 1,
 'src_data': 'mimiciii',
 'task': 'mortality',
 'transfer': False,
 'valid_subsets': ['valid', 'test'],
 'value_mode': 'D

In [ ]:
#Fine-tune a pre-trained DescEmb model on the actual set of pkl files
#Input path should be the dest path from preprocessing step
#model path should be the checkpoint path from traing step
#source data- mimiciii , task- mortality, embed_model- descemb_bert, model-ehr_model,value_mode- DSVA_DPE
!python main.py \
    --distributed_world_size 1 \
    --input_path /content/drive/MyDrive/DescEmb_Data/data_output_path/mimic  \
    --model_path /content/drive/MyDrive/DescEmb/outputs/2025-04-29/03-03-06/checkpoints/checkpoint_best.pt \
    --load_pretrained_weights \
    --model ehr_model \
    --embed_model descemb_bert \
    --pred_model rnn \
    --src_data mimiciii \
    --ratio 100 \
    --value_mode DSVA_DPE \
    --task mortality

2025-04-28 20:01:07 | INFO numexpr.utils NumExpr defaulting to 2 threads.)))
[2025-04-28 20:01:09,665][trainers.trainer][INFO] - {'batch_size': 128,
 'bert_model': 'bert_tiny',
 'device_ids': [0],
 'disable_validation': False,
 'distributed_world_size': 1,
 'dropout': 0.3,
 'embed_model': 'descemb_bert',
 'enc_embed_dim': 128,
 'enc_hidden_dim': 256,
 'eval_data': None,
 'fold': None,
 'init_bert_params': False,
 'init_bert_params_with_freeze': False,
 'input_path': '/content/drive/MyDrive/DescEmb_Data/data_output_path/mimic',
 'load_pretrained_weights': True,
 'lr': 0.0001,
 'max_event_len': 150,
 'mlm_prob': 0.3,
 'model': 'ehr_model',
 'model_path': '/content/drive/MyDrive/DescEmb/outputs/2025-04-29/03-03-06/checkpoints/checkpoint_best.pt',
 'n_epochs': 1000,
 'patience': 5,
 'pred_embed_dim': 128,
 'pred_hidden_dim': 256,
 'pred_model': 'rnn',
 'ratio': '100',
 'rnn_layer': 1,
 'save_dir': 'checkpoints',
 'save_prefix': 'checkpoint',
 'seed': 1,
 'src_data': 'mimiciii',
 'task': 'm

In [ ]:
#Pre-train a CodeEmb model with Word2Vec
!python main.py --distributed_world_size 1 --input_path /content/drive/MyDrive/DescEmb_Data/data_output_path/mimic --src_data mimiciii --task w2v --model codeemb

2025-04-28 21:29:12 | INFO numexpr.utils NumExpr defaulting to 2 threads.)))
[2025-04-28 21:29:28,490][trainers.word2vec_trainer][INFO] - epoch: 0, loss: 1.386
[2025-04-28 21:29:28,491][trainers.word2vec_trainer][INFO] - Saving checkpoint to checkpoints/checkpoint_best.pt
[2025-04-28 21:29:28,505][trainers.word2vec_trainer][INFO] - Finished saving checkpoint to checkpoints/checkpoint_best.pt
[2025-04-28 21:29:28,658][trainers.word2vec_trainer][INFO] - epoch: 1, loss: 1.386
Validation AUROC increased (0.000000 --> -1.386215)
[2025-04-28 21:29:28,658][trainers.word2vec_trainer][INFO] - Saving checkpoint to checkpoints/checkpoint_best.pt
[2025-04-28 21:29:28,670][trainers.word2vec_trainer][INFO] - Finished saving checkpoint to checkpoints/checkpoint_best.pt
[2025-04-28 21:29:28,824][trainers.word2vec_trainer][INFO] - epoch: 2, loss: 1.386
Validation AUROC increased (-1.386215 --> -1.386113)
[2025-04-28 21:29:28,824][trainers.word2vec_trainer][INFO] - Saving checkpoint to checkpoints/check

In [ ]:
#Train a new codeemb model with higher patience parameter to allow more epochs for vlauemode DSVA_DPE
#Input path should be the dest path from preprocessing step
#source data- mimiciii , task- mortality, embed_model- codeemb, model-ehr_model,value_mode- DSVA_DPE
!python main.py \
    --distributed_world_size 1 \
    --input_path /content/drive/MyDrive/DescEmb_Data/data_output_path/mimic \
    --model ehr_model \
    --embed_model codeemb \
    --pred_model rnn \
    --src_data mimiciii \
    --ratio 100 \
    --value_mode DSVA_DPE \
    --task mortality \
    --patience 20 ;

2025-04-28 21:38:23 | INFO numexpr.utils NumExpr defaulting to 2 threads.)))
[2025-04-28 21:38:25,216][trainers.trainer][INFO] - {'batch_size': 128,
 'bert_model': 'bert_tiny',
 'device_ids': [0],
 'disable_validation': False,
 'distributed_world_size': 1,
 'dropout': 0.3,
 'embed_model': 'codeemb',
 'enc_embed_dim': 128,
 'enc_hidden_dim': 256,
 'eval_data': None,
 'fold': None,
 'init_bert_params': False,
 'init_bert_params_with_freeze': False,
 'input_path': '/content/drive/MyDrive/DescEmb_Data/data_output_path/mimic',
 'load_pretrained_weights': False,
 'lr': 0.0001,
 'max_event_len': 150,
 'mlm_prob': 0.3,
 'model': 'ehr_model',
 'model_path': None,
 'n_epochs': 1000,
 'patience': 20,
 'pred_embed_dim': 128,
 'pred_hidden_dim': 256,
 'pred_model': 'rnn',
 'ratio': '100',
 'rnn_layer': 1,
 'save_dir': 'checkpoints',
 'save_prefix': 'checkpoint',
 'seed': 1,
 'src_data': 'mimiciii',
 'task': 'mortality',
 'transfer': False,
 'valid_subsets': ['valid', 'test'],
 'value_mode': 'DSVA_D

In [ ]:
#Train a new codeemb model with higher patience parameter to allow more epochs for valuemode DSVA
#Input path should be the dest path from preprocessing step
#source data- mimiciii , task- mortality, embed_model- codeemb, model-ehr_model,value_mode- DSVA
!python main.py \
    --distributed_world_size 1 \
    --input_path /content/drive/MyDrive/DescEmb_Data/data_output_path/mimic \
    --model ehr_model \
    --embed_model codeemb \
    --pred_model rnn \
    --src_data mimiciii \
    --ratio 100 \
    --value_mode DSVA \
    --task mortality \
    --patience 20 ;

2025-04-28 21:47:31 | INFO numexpr.utils NumExpr defaulting to 2 threads.)))
[2025-04-28 21:47:33,076][trainers.trainer][INFO] - {'batch_size': 128,
 'bert_model': 'bert_tiny',
 'device_ids': [0],
 'disable_validation': False,
 'distributed_world_size': 1,
 'dropout': 0.3,
 'embed_model': 'codeemb',
 'enc_embed_dim': 128,
 'enc_hidden_dim': 256,
 'eval_data': None,
 'fold': None,
 'init_bert_params': False,
 'init_bert_params_with_freeze': False,
 'input_path': '/content/drive/MyDrive/DescEmb_Data/data_output_path/mimic',
 'load_pretrained_weights': False,
 'lr': 0.0001,
 'max_event_len': 150,
 'mlm_prob': 0.3,
 'model': 'ehr_model',
 'model_path': None,
 'n_epochs': 1000,
 'patience': 20,
 'pred_embed_dim': 128,
 'pred_hidden_dim': 256,
 'pred_model': 'rnn',
 'ratio': '100',
 'rnn_layer': 1,
 'save_dir': 'checkpoints',
 'save_prefix': 'checkpoint',
 'seed': 1,
 'src_data': 'mimiciii',
 'task': 'mortality',
 'transfer': False,
 'valid_subsets': ['valid', 'test'],
 'value_mode': 'DSVA'}

In [ ]:
#Fine-tune a pre-trained codeemb model
#Input path should be the dest path from preprocessing step
#model path should be the checkpoint path from traing step
#source data- mimiciii , task- mortality, embed_model- codeemb, model-ehr_model,value_mode- DSVA
!python main.py \
    --distributed_world_size 1 \
    --input_path /content/drive/MyDrive/DescEmb_Data/data_output_path/mimic  \
    --model_path /content/drive/MyDrive/DescEmb/outputs/2025-04-29/06-47-33/checkpoints/checkpoint_best.pt \
    --load_pretrained_weights \
    --model ehr_model \
    --embed_model codeemb \
    --pred_model rnn \
    --src_data mimiciii \
    --ratio 100 \
    --value_mode DSVA \
    --task mortality

2025-04-28 22:33:46 | INFO numexpr.utils NumExpr defaulting to 2 threads.)))
[2025-04-28 22:33:49,842][trainers.trainer][INFO] - {'batch_size': 128,
 'bert_model': 'bert_tiny',
 'device_ids': [0],
 'disable_validation': False,
 'distributed_world_size': 1,
 'dropout': 0.3,
 'embed_model': 'codeemb',
 'enc_embed_dim': 128,
 'enc_hidden_dim': 256,
 'eval_data': None,
 'fold': None,
 'init_bert_params': False,
 'init_bert_params_with_freeze': False,
 'input_path': '/content/drive/MyDrive/DescEmb_Data/data_output_path/mimic',
 'load_pretrained_weights': True,
 'lr': 0.0001,
 'max_event_len': 150,
 'mlm_prob': 0.3,
 'model': 'ehr_model',
 'model_path': '/content/drive/MyDrive/DescEmb/outputs/2025-04-29/06-47-33/checkpoints/checkpoint_best.pt',
 'n_epochs': 1000,
 'patience': 5,
 'pred_embed_dim': 128,
 'pred_hidden_dim': 256,
 'pred_model': 'rnn',
 'ratio': '100',
 'rnn_layer': 1,
 'save_dir': 'checkpoints',
 'save_prefix': 'checkpoint',
 'seed': 1,
 'src_data': 'mimiciii',
 'task': 'mortal

In [ ]:
import torch

checkpoint = torch.load('/content/drive/MyDrive/DescEmb/outputs/2025-04-29/06-47-33/checkpoints/checkpoint_best.pt', weights_only=False)
print(checkpoint.keys())


dict_keys(['model_state_dict', 'optimizer_state_dict', 'epochs', 'args'])


In [ ]:
for key in checkpoint['model_state_dict'].keys():
    print(key)


embed_model.embedding.weight
pred_model.model.weight_ih_l0
pred_model.model.weight_hh_l0
pred_model.model.bias_ih_l0
pred_model.model.bias_hh_l0
pred_model.final_proj.weight
pred_model.final_proj.bias


In [14]:
#Transfer a trained model from mimiciii to eicu using batch size 16 to avoid memory load error
!python main.py \
    --distributed_world_size 1 \
    --input_path /content/drive/MyDrive/DescEmb/data_output_path/ \
    --model_path /content/drive/MyDrive/DescEmb/outputs/2025-04-29/03-03-06/checkpoints/checkpoint_best.pt \
    --model ehr_model \
    --embed_model descemb_bert \
    --pred_model rnn \
    --src_data eicu \
    --ratio 100 \
    --value_mode DSVA_DPE \
    --task mortality \
    --batch_size 16;

2025-04-29 03:38:29 | INFO numexpr.utils NumExpr defaulting to 8 threads.)))
[2025-04-29 03:38:31,047][trainers.trainer][INFO] - {'batch_size': 16,
 'bert_model': 'bert_tiny',
 'device_ids': [0],
 'disable_validation': False,
 'distributed_world_size': 1,
 'dropout': 0.3,
 'embed_model': 'descemb_bert',
 'enc_embed_dim': 128,
 'enc_hidden_dim': 256,
 'eval_data': None,
 'fold': None,
 'init_bert_params': False,
 'init_bert_params_with_freeze': False,
 'input_path': '/content/drive/MyDrive/DescEmb/data_output_path/',
 'load_pretrained_weights': False,
 'lr': 0.0001,
 'max_event_len': 150,
 'mlm_prob': 0.3,
 'model': 'ehr_model',
 'model_path': '/content/drive/MyDrive/DescEmb/outputs/2025-04-29/03-03-06/checkpoints/checkpoint_best.pt',
 'n_epochs': 1000,
 'patience': 5,
 'pred_embed_dim': 128,
 'pred_hidden_dim': 256,
 'pred_model': 'rnn',
 'ratio': '100',
 'rnn_layer': 1,
 'save_dir': 'checkpoints',
 'save_prefix': 'checkpoint',
 'seed': 1,
 'src_data': 'eicu',
 'task': 'mortality',
 't

In [ ]:
# generate small set of pkl files for debugging purpse
import pickle

# Paths
input_df_path = '/content/drive/MyDrive/DescEmb_Data/data_output_path/mimic/mimiciii_df.pkl'
input_cohort_path = '/content/drive/MyDrive/DescEmb_Data/data_output_path/mimic/mimiciii_cohort.pkl'

output_df_path = '/content/drive/MyDrive/DescEmb_Data/data_output_path/mimic/small_mimiciii_df.pkl'
output_cohort_path = '/content/drive/MyDrive/DescEmb_Data/data_output_path/mimic/small_mimiciii_cohort.pkl'

# Load full df
with open(input_df_path, 'rb') as f:
    full_df = pickle.load(f)

# Load full cohort
with open(input_cohort_path, 'rb') as f:
    full_cohort = pickle.load(f)

# Slice first N entries (example: first 100 records)
N = 100

small_df = full_df[:N]
small_cohort = full_cohort[:N]

# Save smaller versions
with open(output_df_path, 'wb') as f:
    pickle.dump(small_df, f)

with open(output_cohort_path, 'wb') as f:
    pickle.dump(small_cohort, f)

print(f"Saved sliced datasets with {N} entries each!")
